<a href="https://colab.research.google.com/github/ishammansoor/AI-and-Machine-Learning/blob/main/Emotional_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
df = pd.read_csv("/content/text.csv")

Spliting into traning and testing sets

In [4]:
X=df['text']
y=df['label']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

Creating tokenizing the words using tensorflow and also padding it

In [5]:
tokenizer = Tokenizer(num_words = 393822, oov_token='')
tokenizer.fit_on_texts(X_train)
tokenizer.fit_on_texts(X_test)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

In [6]:
maxlen = max(len(tokens) for tokens in X_train_sequences)
print("Maximum sequence length (maxlen):", maxlen)
X_train_padded = pad_sequences(X_train_sequences, maxlen=maxlen, padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=maxlen, padding='post')

input_size = np.max(X_train_padded) + 1
input_size

Maximum sequence length (maxlen): 178


68487

In [11]:
model = keras.Sequential([
   layers.Embedding(input_dim=input_size, output_dim=100,input_shape=(178,)),
   layers.Bidirectional(layers.LSTM(128)),
   layers.BatchNormalization(),
   layers.Dropout(rate=0.5),
   layers.Dense(64, activation="relu"),
   layers.Dropout(rate=0.5),
   layers.Dense(64, activation="relu"),
   layers.Dropout(rate=0.5),
   layers.Dense(64, activation="relu"),
   layers.Dropout(rate=0.5),
   layers.Dense(6, activation="softmax"),
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [10]:
predictions = model.fit(
    X_train_padded,
    y_train,
    epochs=3,
    batch_size=32,
    validation_data=(X_test_padded, y_test),
    callbacks=[keras.callbacks.EarlyStopping(patience=3)]
)


Epoch 1/3
10421/10421 [==============================] - 224s 21ms/step - loss: 0.2509 - accuracy: 0.9003 - val_loss: 0.1014 - val_accuracy: 0.9368
Epoch 2/3
10421/10421 [==============================] - 203s 19ms/step - loss: 0.1082 - accuracy: 0.9377 - val_loss: 0.0939 - val_accuracy: 0.9389
Epoch 3/3
10421/10421 [==============================] - 194s 19ms/step - loss: 0.1000 - accuracy: 0.9390 - val_loss: 0.0944 - val_accuracy: 0.9395
